In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
from pyspark.sql.functions import col, lag, lead
from pyspark.sql.window import Window
# from mock.tasks import adiciona_carro
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [24]:
session.execute("USE simulacao")

In [25]:
session.execute("select count(*) from simulacao")[0]

/tmp/ipykernel_1363/795349805.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  session.execute("select count(*) from simulacao")[0]


Row(count=3134)

In [26]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 142 ALLOW FILTERING;")

In [27]:
result[0]

/tmp/ipykernel_1363/223460175.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  result[0]


Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ARG5B67', pos_x=655.0, pos_y=168.99999999999872)

In [28]:
r = list(result)

In [29]:
r

[Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ARG5B67', pos_x=655.0, pos_y=168.99999999999872),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='CHI6S78', pos_x=745.0, pos_y=820.0),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ECU6I78', pos_x=565.0, pos_y=90.75),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='GUY1X23', pos_x=385.0, pos_y=310.25000000000017),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='PAR7P89', pos_x=475.0, pos_y=152.25000000000003),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='SUR8W90', pos_x=745.0, pos_y=749.15),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='URU0A12', pos_x=835.0, pos_y=650.5),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='URU4O56', pos_x=565.0, pos_y=869.2500000000007)]

In [30]:
df = ss.createDataFrame(r)

df.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               142|ARG5B67|655.0|168.99999999999872|
| BR-116|               142|CHI6S78|745.0|             820.0|
| BR-116|               142|ECU6I78|565.0|             90.75|
| BR-116|               142|GUY1X23|385.0|310.25000000000017|
| BR-116|               142|PAR7P89|475.0|152.25000000000003|
| BR-116|               142|SUR8W90|745.0|            749.15|
| BR-116|               142|URU0A12|835.0|             650.5|
| BR-116|               142|URU4O56|565.0| 869.2500000000007|
+-------+------------------+-------+-----+------------------+



In [31]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 143 ALLOW FILTERING;")
r = list(result)
df2 = ss.createDataFrame(r)
df2.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               143|ARG5B67|655.0|162.99999999999872|
| BR-116|               143|CHI6S78|745.0|             776.0|
| BR-116|               143|ECU6I78|565.0|325.35000000000014|
| BR-116|               143|GUY1X23|385.0| 435.1000000000003|
| BR-116|               143|PAR7P89|475.0| 268.8000000000002|
| BR-116|               143|SUR8W90|745.0|            749.15|
| BR-116|               143|URU0A12|835.0|             614.5|
| BR-116|               143|URU4O56|565.0| 904.5000000000007|
| BR-116|               143|URU6M78|655.0|            826.75|
+-------+------------------+-------+-----+------------------+



In [32]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 144 ALLOW FILTERING;")
r = list(result)
df25 = ss.createDataFrame(r)
df25.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               144|ARG5B67|655.0|146.99999999999872|
| BR-116|               144|CHI6S78|745.0|             776.0|
| BR-116|               144|ECU6I78|565.0|245.55000000000007|
| BR-116|               144|GUY1X23|385.0|459.75000000000034|
| BR-116|               144|PAR7P89|475.0| 317.5500000000002|
| BR-116|               144|SUR8W90|745.0|             753.5|
| BR-116|               144|URU0A12|835.0|             658.5|
| BR-116|               144|URU4O56|565.0| 800.5500000000006|
| BR-116|               144|URU6M78|745.0|            800.75|
+-------+------------------+-------+-----+------------------+



In [65]:
df_empty = ss.createDataFrame([], "placa: string, posicao: int, faixa: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean, tempo_em_curso: int")

In [66]:
import json
params = json.load(open('./mock/parametros.json'))

In [67]:
p = [[key]+list(params[key].values()) for key in params.keys()]

In [68]:
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))

p.toPandas()

,Rodovia,sentido1Name,sentido2Name,sentido1Faixas,sentido2Faixas,probabilidadeDeEntradaDeVeiculo,velocidadeMaxima,velocidadeMinima,AceleracaoMaxima,AceleracaoMinima,probabilidadeDeTrocaDeFaixa,probabilidadeColisao,tempoColisao,kilometroInicial,interfaceGrafica,tamanhoCarro
0,BR-040,Brasilia,Rio de Janeiro,3,3,0.9,10,2,0.10,0.01,0.3,0.1,3000,100,True,74
1,BR-116,Fortaleza,Rio Grande do Sul,3,3,0.8,9,2,0.15,0.05,0.5,0.1,3000,100,False,74
2,BR-135,Belo Horizonte,São Luís,3,3,0.9,12,3,0.20,0.05,0.3,0.1,3000,100,False,74
3,BR-393,Cachoeiro de Itapemirim,Barra Mansa,2,2,0.9,8,2,0.10,0.01,0.5,0.1,3000,100,False,74


In [69]:
import queue


In [76]:
fps = 30
vel_media = 0
n_vel_media = 0

tempo_medio = 0
n_tempo_medio = 0

def atualiza_media(media_atual, tamanho_atual, media_add, tamanho_add):
    if media_add == None:
        return media_atual
    if tamanho_atual == 0:
        return media_add
    tamanho_total = tamanho_atual + tamanho_add
    return (media_atual/tamanho_total)*tamanho_atual + (media_add/tamanho_total)*tamanho_add

def processa_velocidade_media(batch):
    global vel_media, n_vel_media
    batch = batch.na.fill(0, subset=['velocidade'])
    vel_media_batch = batch.agg(F.mean('velocidade')).collect()[0][0]
    length_batch = batch.select(F.count('velocidade')).collect()[0][0]
    n_vel_media += length_batch
    vel_media = atualiza_media(vel_media, n_vel_media, vel_media_batch, length_batch)
    pass

def processa_tempo_cruzamento(batch):
    global n_tempo_medio, tempo_medio
    batch = batch.filter(col("tempo_em_curso") != 0)
    tempo_medio_batch = batch.agg(F.mean('tempo_em_curso')).collect()[0][0]
    length_batch = batch.select(F.count('tempo_em_curso')).collect()[0][0]
    n_tempo_medio += length_batch
    tempo_medio = atualiza_media(tempo_medio, n_tempo_medio, tempo_medio_batch, length_batch)
    pass

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    DadosCarros = DadosCarros.drop('aplicaMulta')

    # Renomeia coluna do dado novo
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia').alias('rodovia_nova'),
                                   F.col('pos_x').alias('faixa_nova'),
                                   F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')
    
    CarrosSumidos = DadosCarros.join(DadosNovos, on="placa", how='left_anti')
    processa_tempo_cruzamento(CarrosSumidos)
    data_joined = data_joined.withColumn("tempo_inicio", when(col("tempo_da_simulacao").isNull(), col("tempo_da_simulacao_novo")).otherwise(col("tempo_da_simulacao")))
    data_joined = data_joined.withColumn("tempo_em_curso", col("tempo_da_simulacao_novo") - col("tempo_inicio"))
    data_joined = data_joined.withColumn("rodovia", coalesce(col("rodovia_nova"), col("rodovia")))
    data_joined = data_joined.withColumn("faixa", coalesce(col("faixa_nova"), col("faixa")))
    data_joined = data_joined.drop("rodovia_nova", "faixa_nova")
    
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)

    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")

    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))
    processa_velocidade_media(data_joined)
    
    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))

    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))

    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")

    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")

    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")

    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))

    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )

    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')

    data_joined = data_joined.drop("VelocidadeMaxima")

    window_spec = Window.partitionBy("rodovia", "faixa").orderBy('posicao')
    
    # Use lag function with the window specification
    lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec)
    lead_column = lead(col("posicao_prevista")).over(window_spec) - col("posicao_prevista")

    
    # Add the lag column to the DataFrame
    data_joined = data_joined.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec)) & (col("rodovia") == lag(col("rodovia")).over(window_spec)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec)) & (col("rodovia") == lead(col("rodovia")).over(window_spec))), 1).otherwise(0))
    
    # Show the result
    return data_joined
    

In [77]:
df_empty.show()

+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+--------------+
|placa|posicao|faixa|rodovia|tempo_da_simulacao|velocidade|aceleracao|posicao_prevista|acima_vel|aplicaMulta|tempo_em_curso|
+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+--------------+
+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+--------------+



In [78]:
df3 = processa_carro(df, df_empty, 1, 0.5, p)

In [79]:
df3 = processa_carro(df2, df3, 1, 0.5, p)

In [80]:
df4 = processa_carro(df25, df3, 1, 0.5, p)

In [81]:
df4.show()

+-------+-------+-----+--------------------+------------------+---------+--------------+------------+-------------+------------------+------------------+-------------------+-----------+
|  placa|rodovia|faixa|          aceleracao|  posicao_prevista|acima_vel|tempo_em_curso|tempo_inicio|Risco_Colisão|           posicao|tempo_da_simulacao|         velocidade|aplicaMulta|
+-------+-------+-----+--------------------+------------------+---------+--------------+------------+-------------+------------------+------------------+-------------------+-----------+
|GUY1X23| BR-116|385.0|-0.11133333333333346|460.51600000000036|        0|             1|         143|            0|459.75000000000034|               144| 0.8216666666666678|      false|
|PAR7P89| BR-116|475.0| -0.0753333333333335| 319.1373333333335|        0|             1|         143|            0| 317.5500000000002|               144|              1.625|      false|
|ECU6I78| BR-116|565.0| -0.3493333333333336| 242.7153333333334|       

In [ ]:
vel_media